In [1]:
import json 
import numpy as np 
import open3d as o3d
from extract_context_teeth import extract_teeth_group_and_their_gums, mesh_to_pcd
import os 
from glob import glob
import re

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def extract_segments_from_json(json_file_path):
    with open(json_file_path, 'r') as f:
        data = json.load(f)
    segments = []
    for cell in data['cells']:
        segments.append(cell['fdi'])
    segments = np.array(segments)
    try:
        abutment_teeth = data["abutment_teeth"]
    except KeyError:
        abutment_teeth = None
        print(f"Warning: No 'abutment_teeth' key in {json_file_path}")

    return segments, abutment_teeth

In [3]:
ply_names = sorted(glob("/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/CROWN-PARTIAL-ANNOTATED-TOOTH/ply/*.ply"))
json_names = sorted(glob("/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/CROWN-PARTIAL-ANNOTATED-TOOTH/json/*.json"))

elements = []
for ply_name, json_name in zip(ply_names, json_names):
    # print(ply_name)
    # print(json_name)

    match = re.search(r"CROWN-PARTIAL-([0-9]{4}-[0-9]{2}-[0-9]{2}_[0-9]{5}-\d{3}_[0-9]{4}-[0-9]{2}-[0-9]{2}_[0-9]{5}-\d{3})",json_name).group(1)
    
    # print(match)

    segment, abutment_teeth = extract_segments_from_json(json_name)
    if abutment_teeth:
        # print(json_name)
        # print(abutment_teeth)
        # print("-"*80)
        elements.append(abutment_teeth[0])

In [4]:
# elements

In [5]:
set(elements)

{36, 46}

In [6]:
# Why instead of just one neighbor, we try to keep instances for two neighbors, it is because for example we want to extract left neighbor of tooth 36(molar), it is 35(premolar) but in edge case some people wouldn't have two premolar, and only one premolar(34 or 35)
# In such case we take backup teeth 35 which is its nearest left neighbor 

abut_context_teeth_dict_ = {
    "abutment_teeth":["Left_Neighbor", "Abutment Teeth" ,"Right_Neighbor", "Left_Antagonist", "Middle_Antagonist", "Right_Antagonist"],
    32: [[31,41], [32], [33,34], [21,11], [22], [23,24]], 
    35: [[34,33], [35], [36,37], [24,23], [25], [26,27]],
    36: [[35,34], [36], [37,38], [25,24], [26], [27,28]],
    37: [[36,35], [37], [38], [26,25], [27], [28]],

    44: [[45,46], [44], [43,42], [15,16], [14], [13,12]],
    46: [[47,48], [46], [45,44], [17, 18], [16], [15, 14]],
    47: [[48], [47], [46,45], [18], [17], [16,15]],
}

In [7]:
def extract_highest_priority(tooth_number, available_teeth, abut_dict):
    result = []
    for group in abut_dict[tooth_number]:
        selected = None
        for candidate in group:
            if candidate in available_teeth:
                selected = candidate
                break  # stop at the first match (highest priority)
        if selected is not None:
            result.append(selected)
    return result

In [8]:
# available_teeth = np.unique(segment)

# target_segment_ids = extract_highest_priority(abutment_teeth[0], available_teeth, abut_context_teeth_dict_)
# target_segment_ids

In [9]:
ply_names = sorted(glob("/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/CROWN-PARTIAL-ANNOTATED-TOOTH/ply/*.ply"))
json_names = sorted(glob("/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/CROWN-PARTIAL-ANNOTATED-TOOTH/json/*.json"))

elements = []
for ply_name, json_name in zip(ply_names, json_names):
    # print(ply_name)
    # print(json_name)
    segment_only_abutment, abutment_teeth = extract_segments_from_json(json_name)
    if abutment_teeth:
        # print(json_name)
        # print(abutment_teeth)
        # print("-"*80)
        # elements.append(abutment_teeth[0])

        # print(abut_context_teeth_dict_[abutment_teeth[0]])

        label = "master"
        mesh = o3d.io.read_triangle_mesh(ply_name)

        available_teeth = np.unique(segment_only_abutment) # A SET OF AVAILABLE LABELS WHICH WE HAVE TO CHOOSE FROM 
        target_segment_ids = extract_highest_priority(abutment_teeth[0], available_teeth, abut_context_teeth_dict_)

        # print(available_teeth)
        # print(target_segment_ids)

        abutment_mesh, _, _ = extract_teeth_group_and_their_gums(mesh, segment_only_abutment, target_segment_ids)
        pcd_abutment_mesh, npy_abutment_mesh = mesh_to_pcd(abutment_mesh, num_points=1000)

        unique_customer_id = re.search(r"CROWN-PARTIAL-([0-9]{4}-[0-9]{2}-[0-9]{2}_[0-9]{5}-\d{3}_[0-9]{4}-[0-9]{2}-[0-9]{2}_[0-9]{5}-\d{3})", os.path.basename(json_name)).group(1)
        
        if abutment_teeth[0] in [11,12,13,14,15,16,17,18,21,22,23,24,25,26,27,28]: # UPPER JAW 
            new_MASTER_ply_name = os.path.join(os.path.dirname(ply_name), "CROWN-PARTIAL-" + unique_customer_id + "-upperjaw.ply")
            new_MASTER_json_name = os.path.join(os.path.dirname(json_name), "CROWN-PARTIAL-" + unique_customer_id + "-upperjaw.json")

            new_ANTAGONIST_ply_name = os.path.join(os.path.dirname(ply_name), "CROWN-PARTIAL-" + unique_customer_id + "-lowerjaw.ply")
            new_ANTAGONIST_json_name = os.path.join(os.path.dirname(json_name), "CROWN-PARTIAL-" + unique_customer_id + "-lowerjaw.json")

        
        elif abutment_teeth[0] in [31,32,33,34,35,36,37,38,41,42,43,44,45,46,47,48]: #LOWER JAW 

            new_MASTER_ply_name = os.path.join(os.path.dirname(ply_name), "CROWN-PARTIAL-" + unique_customer_id + "-lowerjaw.ply")
            new_MASTER_json_name = os.path.join(os.path.dirname(json_name), "CROWN-PARTIAL-" + unique_customer_id + "-lowerjaw.json")

            new_ANTAGONIST_ply_name = os.path.join(os.path.dirname(ply_name), "CROWN-PARTIAL-" + unique_customer_id + "-upperjaw.ply")
            new_ANTAGONIST_json_name = os.path.join(os.path.dirname(json_name), "CROWN-PARTIAL-" + unique_customer_id + "-upperjaw.json")

        # print(new_MASTER_ply_name)
        # print(new_ANTAGONIST_ply_name)
        
        segment, _ = extract_segments_from_json(new_MASTER_json_name)
        mesh = o3d.io.read_triangle_mesh(new_MASTER_ply_name)

        available_teeth = np.unique(segment) # A SET OF AVAILABLE LABELS WHICH WE HAVE TO CHOOSE FROM 
        target_segment_ids = extract_highest_priority(abutment_teeth[0], available_teeth, abut_context_teeth_dict_)

        MASTER_mesh, _, _ = extract_teeth_group_and_their_gums(mesh, segment, target_segment_ids)
        pcd_master_mesh, npy_master_mesh = mesh_to_pcd(MASTER_mesh, num_points=3096)


        #COMBINE ABUTMENT TOOTH PCD MESH AND MASTER MESH
        combined_pcd_MASTER_mesh = pcd_abutment_mesh + pcd_master_mesh

        os.makedirs(f"outputs-CROWN-PARTIAL-ANNOTATED/{unique_customer_id}", exist_ok=True)
        o3d.io.write_point_cloud(os.path.join(f"outputs-CROWN-PARTIAL-ANNOTATED/{unique_customer_id}", f"{unique_customer_id}_{label}.pcd"), combined_pcd_MASTER_mesh)
        print(os.path.join(f"outputs-CROWN-PARTIAL-ANNOTATED/{unique_customer_id}", f"{unique_customer_id}_{label}.pcd"))


        # FOR OPPOSITE JAW  
        segment, _ = extract_segments_from_json(new_ANTAGONIST_json_name)
        label = "Antagonist"
        mesh = o3d.io.read_triangle_mesh(new_ANTAGONIST_ply_name)

        available_teeth = np.unique(segment) # A SET OF AVAILABLE LABELS WHICH WE HAVE TO CHOOSE FROM 
        target_segment_ids = extract_highest_priority(abutment_teeth[0], available_teeth, abut_context_teeth_dict_)

        ANTAGONIST_mesh, _, _ = extract_teeth_group_and_their_gums(mesh, segment, target_segment_ids)
        pcd_antagonist_mesh, npy_antagonist_mesh = mesh_to_pcd(ANTAGONIST_mesh, num_points=4096)

        o3d.io.write_point_cloud(os.path.join(f"outputs-CROWN-PARTIAL-ANNOTATED/{unique_customer_id}", f"{unique_customer_id}_{label}.pcd"), pcd_antagonist_mesh)
        print(os.path.join(f"outputs-CROWN-PARTIAL-ANNOTATED/{unique_customer_id}", f"{unique_customer_id}_{label}.pcd"))
    else:
        continue

Original mesh: 37012 vertices, 73522 triangles
Found 1 connected components
Component sizes (triangles): [73522]
Keeping 1 components with >= 5290 triangles
Cleaned mesh: 37012 vertices, 73522 triangles
Original mesh: 24902 vertices, 48975 triangles
Found 2 connected components
Component sizes (triangles): [33008 15967]
Keeping 2 components with >= 5290 triangles
Cleaned mesh: 24902 vertices, 48975 triangles
outputs-CROWN-PARTIAL-ANNOTATED/2018-10-16_00001-021_2018-10-16_00001-021/2018-10-16_00001-021_2018-10-16_00001-021_master.pcd
Original mesh: 28269 vertices, 55678 triangles
Found 1 connected components
Component sizes (triangles): [55678]
Keeping 1 components with >= 5290 triangles
Cleaned mesh: 28269 vertices, 55678 triangles
outputs-CROWN-PARTIAL-ANNOTATED/2018-10-16_00001-021_2018-10-16_00001-021/2018-10-16_00001-021_2018-10-16_00001-021_Antagonist.pcd
Original mesh: 42300 vertices, 84009 triangles
Found 1 connected components
Component sizes (triangles): [84009]
Keeping 1 comp

In [ ]:
ply_names = sorted(glob("/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/CROWN-PARTIAL-ANNOTATED-TOOTH/ply/*.ply"))
json_names = sorted(glob("/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/CROWN-PARTIAL-ANNOTATED-TOOTH/json/*.json"))

elements = []
for ply_name, json_name in zip(ply_names, json_names):
    # print(ply_name)
    # print(json_name)
    segment_only_abutment, abutment_teeth = extract_segments_from_json(json_name)
    if abutment_teeth:
        continue
    else: 
        if "upperjaw" not in os.path.basename(json_name) and "lowerjaw" not in os.path.basename(json_name):
            print(os.path.basename(json_name))


CROWN-PARTIAL-2018-10-24_00001-005_2018-10-24_00001-005-46.json


In [11]:
# THIS TOOTH IS PROBLAMATIC
# CROWN-PARTIAL-2018-10-24_00001-005_2018-10-24_00001-005-46


In [12]:
print("hello")

hello


### For PCD Extraction of PARTIAL-Crown ONLY

In [2]:
ply_dir_names = sorted(glob("/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/outputs-CROWN-PARTIAL-ANNOTATED/*/"))

print(ply_dir_names[0].split('/')[-2])
print(ply_dir_names[0])

2018-10-16_00001-021_2018-10-16_00001-021
/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/outputs-CROWN-PARTIAL-ANNOTATED/2018-10-16_00001-021_2018-10-16_00001-021/


In [3]:
ply_crown_names = sorted(glob("/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/Crown-PARTIAL-Only/*/*"))
ply_crown_names

['/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/Crown-PARTIAL-Only/2018-10-16_00001-021/2018-10-16_00001-021-36-crown_cad.stl',
 '/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/Crown-PARTIAL-Only/2018-10-16_00001-022/2018-10-16_00001-022-36-crown_cad.stl',
 '/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/Crown-PARTIAL-Only/2018-10-16_00001-023/2018-10-16_00001-023-36-crown_cad.stl',
 '/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/Crown-PARTIAL-Only/2018-10-16_00001-024/2018-10-16_00001-024-36-crown_cad.stl',
 '/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/Crown-PARTIAL-Only/2018-10-16_00002-019/2018-10-16_00002-019-36-crown_cad.stl',
 '/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/Crown-PARTIAL-Only/2018-10-16_00002-020/2018-10-16_00002-020-36-crown_cad.stl',
 '/home/shirshak/Teeth_3DS_data_preparation_for_reconstuct

In [4]:
for ply_crown_name in ply_crown_names:
    mesh = o3d.io.read_triangle_mesh(ply_crown_name)
    crown_pcd, crown_npy = mesh_to_pcd(mesh, num_points=1536)
    for ply_dir_name in ply_dir_names: 
        # print(ply_crown_name.split("/")[-2]) 
        # print(ply_dir_name.split('/')[-2])
        if ply_crown_name.split("/")[-2] in ply_dir_name.split('/')[-2]:
            print(os.path.join(ply_dir_name, ply_crown_name.split('/')[-2] + "_shell.pcd"))
            o3d.io.write_point_cloud(os.path.join(ply_dir_name, ply_dir_name.split('/')[-2] + "_shell.pcd"), crown_pcd)
            continue

/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/outputs-CROWN-PARTIAL-ANNOTATED/2018-10-16_00001-021_2018-10-16_00001-021/2018-10-16_00001-021_shell.pcd
/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/outputs-CROWN-PARTIAL-ANNOTATED/2018-10-16_00001-022_2018-10-16_00001-022/2018-10-16_00001-022_shell.pcd
/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/outputs-CROWN-PARTIAL-ANNOTATED/2018-10-16_00001-023_2018-10-16_00001-023/2018-10-16_00001-023_shell.pcd
/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/outputs-CROWN-PARTIAL-ANNOTATED/2018-10-16_00001-024_2018-10-16_00001-024/2018-10-16_00001-024_shell.pcd
/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/outputs-CROWN-PARTIAL-ANNOTATED/2018-10-16_00002-019_2018-10-16_00002-019/2018-10-16_00002-019_shell.pcd
/home/shirshak/Teeth_3DS_data_preparation_for_reconstuction_and_generation/outputs-CROWN-PARTIAL-ANN

In [5]:
print("hello")

hello
